# Weak-lensing galaxy shape catalogue validation

## Cosmology

Contents
- Shear-shear correlation function
- Cluster lensing

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [ ]:
import os

from cs_util.plots import plot_data_1d

In [ ]:
from cs_util import canfar
from sp_validation.util import *
from sp_validation.cat import *
from sp_validation.cosmology import *

### Shear-shear correlation function

In [ ]:
res_g = {}

for sh in shapes:
    res_g[sh]= xi_gal_gal_tc(
        ra[sh],
        dec[sh],
        g_corr_mc[sh][0],
        g_corr_mc[sh][1],
        w[sh],
        ra[sh],
        dec[sh],
        g_corr_mc[sh][0],
        g_corr_mc[sh][1],
        w[sh],
        theta_min_amin=theta_min_amin,
        theta_max_amin=theta_max_amin,
        n_theta=n_theta
    )

In [ ]:
# Redshift distribution

nz_base = 'nz.CFHTLenSmatched.W3'
nz_ext = 'txt'
nz_date = '202012'
nz_version = 'v1'
nz_name = '{}_{}_{}.{}'.format(nz_base, nz_date, nz_version, nz_ext)
source = 'vos:cfis/cosmostat/cosmology/redshifts/{}'.format(nz_name)
target = f'{data_dir}/{nz_name}'

canfar.download(source, target, verbose=True)

z, nz = np.loadtxt(target, unpack=True)

In [ ]:
# Use common angular bins
for sh in shapes:
    r_planck = res_g[sh].meanr
    break

try:
    import pyccl as ccl

    xi_p_planck, xi_m_planck = get_theo_xi(
        r_planck,
        z,
        nz,
        Omega_m=Om,
        h=h,
        Omega_b=Ob,
        sig8=sig8,
        ns=ns
    )

except:
    print("Warning: Cannot impory pyccl")
    xi_p_planck = None
    xi_m_planck = None


In [ ]:
labels = ['CFIS', '', 'Planck model']
linestyles = ['', '', 'solid']
eb_linestyles = ['-', ':', '']
colors = ['b', 'b', 'b']
xlabel = r'$\theta$ [arcmin]'
ylabel = r'$\xi_+(\theta)$'
xlog = True
ylog = True

for sh in shapes:
    pos_ind_gg = res_g[sh].xip >= 0
    neg_ind_gg = res_g[sh].xip < 0

    x = [
        res_g[sh].meanr[pos_ind_gg],
        res_g[sh].meanr[neg_ind_gg],
    ]
    y = [
        res_g[sh].xip[pos_ind_gg],
        -res_g[sh].xip[neg_ind_gg],
    ]
    yerr = [
        np.sqrt(res_g[sh].varxip[pos_ind_gg]),
        np.sqrt(res_g[sh].varxip[neg_ind_gg]),
    ]

    if xi_p_planck is not None:
        x.append(r_planck)
        y.append(xi_p_planck)
        yerr.append(xi_p_planck * np.nan)

    title = fr'{sh} Shear-shear correlation function $\xi_+$'
    out_path = f'{plot_dir}/xi_+_shear_shear_{sh}.pdf'

    plot_data_1d(
        x,
        y,
        yerr,
        title,
        xlabel,
        ylabel,
        out_path,
        xlog=xlog,
        ylog=ylog,
        labels=labels,
        colors=colors,
        linestyles=linestyles,
        eb_linestyles=eb_linestyles
    )

In [ ]:
colors = ['g', 'g', 'g']
ylabel = r'$\xi_-(\theta)$'

for sh in shapes:
    pos_ind_gg = res_g[sh].xim >= 0
    neg_ind_gg = res_g[sh].xim < 0

    x = [
        res_g[sh].meanr[pos_ind_gg],
        res_g[sh].meanr[neg_ind_gg],
    ]
    y = [
        res_g[sh].xim[pos_ind_gg],
        -res_g[sh].xim[neg_ind_gg],
    ]
    yerr = [
        np.sqrt(res_g[sh].varxim[pos_ind_gg]),
        np.sqrt(res_g[sh].varxim[neg_ind_gg]),
    ]
    title = f'{sh} Shear-shear correlation function $\\xi_-$'
    out_path = f'{plot_dir}/xi_-_shear_shear_{sh}.pdf'

    if xi_m_planck is not None:
        x.append(r_planck)
        y.append(xi_m_planck)
        yerr.append(xi_m_planck * np.nan)
        
    plot_data_1d(
        x,
        y,
        yerr,
        title,
        xlabel,
        ylabel,
        out_path,
        xlog=xlog,
        ylog=ylog,
        labels=labels,
        colors=colors,
        linestyles=linestyles,
        eb_linestyles=eb_linestyles
    )

### Cluster lensing

#### Prepare Planck cluster catalog

In [ ]:
cluster_cat_name = 'HFI_PCCS_SZ-union_R2.08.fits.gz'
vos_dir = 'vos:cfis/cosmostat/cosmology/external/Planck'

clusters = get_clusters(cluster_cat_name, vos_dir, data_dir, name)

print_stats(
    f"{len(clusters['ra'])} clusters found in {name} footprint",
    stats_file,
    verbose=verbose
)

In [ ]:
for sh in shapes:

    plt.figure(figsize=(15,15))
    plt.plot(ra[sh], dec[sh], '.')
    plt.plot(clusters['ra'], clusters['dec'], '*')

    dy = 0.02 * (plt.ylim()[0] - plt.ylim()[1])
    for i in range(len(clusters['ra'])):
        plt.text(
            clusters['ra'][i],
            clusters['dec'][i] + dy,
            i,
            color='k',
            fontsize=10,
            ha='center',
            va='center'
        )

    plt.title(f'({sh}) clusters in {name} footprint')
    plt.xlabel('R.A. [deg]')
    plt.ylabel('DEC [deg]')

In [ ]:
# TODO: Identify clusters within unmasked footprint.

In [ ]:
ylabel = r'$\gamma_{\rm t}(\theta)$'

for sh in shapes:
 
    R_p, logR_p, gamT_p, gamX_p, gam_sig_p = gamma_T_tc(
        clusters['ra'],
        clusters['dec'],
        ra[sh],
        dec[sh],
        g_corr_mc[sh][0],
        g_corr_mc[sh][1],
        w[sh]
    )

    x = [R_p,R_p]
    y = [gamT_p,gamX_p]
    yerr = [gam_sig_p,gam_sig_p]
    title = f'({sh}) tangential shear around clusters'
    out_path = f'{plot_dir}/gamma_clusters_{sh}.pdf'
    labels=[r'$\gamma_{\rm t}(\theta)$',r'$\gamma_{\rm X}(\theta)$']
    
    plot_data_1d(x, y, yerr, title, xlabel, ylabel, out_path, xlog=True, ylog=False,labels=labels)

## Wrap-up

In [ ]:
# Copy parameter file to output for save-guarding
!cp params.py {output_dir}